In [1]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai
import os, json

c:\Users\dinh0\anaconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Load the review data
data = json.load(open("reviews.json"))

In [7]:
# Embed the data with Gemini AI

processed_data = []

genai.configure(api_key=os.environ["GEMINIAI_API_KEY"])

model = genai.GenerativeModel('gemini-1.5-flash')

# Create embeddings for each review
for review in data["reviews"]:
    review_string = json.dumps(review)
    response = genai.embed_content(
        model="models/text-embedding-004",
        content=review_string
    )
    embedding = response['embedding']
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "subject": review["course"],
                "stars": review["rating"],
                "review": review["reviews"],
                "department": review["department"],
            }
        }
    )

In [4]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [ ]:
# Create a Pinecone index
pc.create_index(
    name="rateprof",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [8]:
# Insert embedded data into Pinecone
index = pc.Index("rateprof")
index.upsert(
    vectors=processed_data,
    namespace = "test1"
)


{'upserted_count': 20}